In [10]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler,StandardScaler, LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix,ConfusionMatrixDisplay,classification_report

In [11]:
df = pd.read_csv("E:\VScodeWorkSheet\datasets\Google-Playstore.csv")
df=df.sample(frac=0.01,replace=True,random_state=1)
print(df.head())

                                                 App Name  \
128037                      Burn it Down Piano Cover Song   
491755                                       أنعام الوسطى   
491263                          Block Puzzle Ultimate Fun   
836489  Learning Professions and Occupations for Toddlers   
73349                                            Finutive   

                                      App Id       Category  Rating  \
128037                 com.just.BurnitLinkin  Entertainment     0.0   
491755                    com.anaamw.anaamap   Food & Drink     4.3   
491263  com.namoideas.blockpuzzleultimatefun         Puzzle     0.0   
836489                       com.gokids.prof    Educational     3.7   
73349                       com.finutive.app       Business     4.3   

        Rating Count  Installs  Minimum Installs  Maximum Installs  Free  \
128037           0.0       10+              10.0                22  True   
491755           9.0    1,000+            1000.0      

In [12]:
for col in df.columns:
    col1 = col.replace(' ','')
    df = df.rename(columns={col:col1})
print(df.columns)

Index(['AppName', 'AppId', 'Category', 'Rating', 'RatingCount', 'Installs',
       'MinimumInstalls', 'MaximumInstalls', 'Free', 'Price', 'Currency',
       'Size', 'MinimumAndroid', 'DeveloperId', 'DeveloperWebsite',
       'DeveloperEmail', 'Released', 'LastUpdated', 'ContentRating',
       'PrivacyPolicy', 'AdSupported', 'InAppPurchases', 'EditorsChoice',
       'ScrapedTime'],
      dtype='object')


In [13]:
print("Dropping the following columns - AppId, DeveloperWebsite, DeveloperEmail, PrivacyPolicy, Currency, DeveloperId, ScrapedTime, MinimumAndroid")
df = df.drop(['AppId','DeveloperWebsite','DeveloperEmail','PrivacyPolicy','Currency','DeveloperId','ScrapedTime','MinimumAndroid'],axis=1)
print(df.head())

Dropping the following columns - AppId, DeveloperWebsite, DeveloperEmail, PrivacyPolicy, Currency, DeveloperId, ScrapedTime, MinimumAndroid
                                                  AppName       Category  \
128037                      Burn it Down Piano Cover Song  Entertainment   
491755                                       أنعام الوسطى   Food & Drink   
491263                          Block Puzzle Ultimate Fun         Puzzle   
836489  Learning Professions and Occupations for Toddlers    Educational   
73349                                            Finutive       Business   

        Rating  RatingCount  Installs  MinimumInstalls  MaximumInstalls  Free  \
128037     0.0          0.0       10+             10.0               22  True   
491755     4.3          9.0    1,000+           1000.0             1086  True   
491263     0.0          0.0       50+             50.0               95  True   
836489     3.7        943.0  500,000+         500000.0           622301  True  

In [14]:
print("Dataset information",df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23129 entries, 128037 to 1416881
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   AppName          23129 non-null  object 
 1   Category         23129 non-null  object 
 2   Rating           22904 non-null  float64
 3   RatingCount      22904 non-null  float64
 4   Installs         23129 non-null  object 
 5   MinimumInstalls  23129 non-null  float64
 6   MaximumInstalls  23129 non-null  int64  
 7   Free             23129 non-null  bool   
 8   Price            23129 non-null  float64
 9   Size             23128 non-null  object 
 10  Released         22467 non-null  object 
 11  LastUpdated      23129 non-null  object 
 12  ContentRating    23129 non-null  object 
 13  AdSupported      23129 non-null  bool   
 14  InAppPurchases   23129 non-null  bool   
 15  EditorsChoice    23129 non-null  bool   
dtypes: bool(4), float64(4), int64(1), object(7)
memory 

In [15]:
cols = df.columns[df.isnull().any()].to_list()
print("Columns having null values are :",cols)

Columns having null values are : ['Rating', 'RatingCount', 'Size', 'Released']


In [16]:
for c in cols:
    print(c,type(c),": ",df[c].isnull().sum())

Rating <class 'str'> :  225
RatingCount <class 'str'> :  225
Size <class 'str'> :  1
Released <class 'str'> :  662


In [17]:
df.dropna(subset=['Size','MinimumInstalls','Installs','AppName'],inplace=True)
df.dropna(subset=['AppName'],inplace=True)

In [18]:
df['Rating']  = df['Rating'].astype(float)
avg = round(df['Rating'].mean(),1)
df['Rating'].fillna(avg,inplace=True)

df['RatingCount']  = df['RatingCount'].astype(float)
avg = round(df['RatingCount'].mean(),1)
df['RatingCount'].fillna(avg,inplace=True)

print(df['ContentRating'].value_counts())

Everyone           20203
Teen                1947
Mature 17+           621
Everyone 10+         354
Adults only 18+        2
Unrated                1
Name: ContentRating, dtype: int64


In [19]:
df['ContentRating'] = df['ContentRating'].replace('Unrated',"Everyone")

#Cleaning other values just to include Everyone, Teens and Adult 

df['ContentRating'] = df['ContentRating'].replace('Mature 17+',"Adults")
df['ContentRating'] = df['ContentRating'].replace('Adults only 18+',"Adults")
df['ContentRating'] = df['ContentRating'].replace('Everyone 10+',"Everyone")

In [20]:
df.Installs = df.Installs.str.replace(',','')
df.Installs = df.Installs.str.replace('+','')
df.Installs = df.Installs.str.replace('Free','0')
df['Installs'] = pd.to_numeric(df['Installs'])
df['Installs'] = df['Installs'].astype(float)

In [21]:
df['PriceRange'] = pd.cut(df['Price'],bins=[0,0.19,9.99,29.99,410],labels=['Free','Low','Mid','High'],include_lowest=True)
print(df['PriceRange'].value_counts())

Free    22678
Low       430
Mid        15
High        5
Name: PriceRange, dtype: int64


In [22]:
print(df.Free.value_counts())
print("Apps that have Price = 0, have Free column True")
df.loc[(df.Price==0) & (df.Free==False),'Free'] = True
print(df.Free.value_counts())

df['Type'] = np.where(df['Free'] == True,'Free','Paid')
df.drop(['Free'],inplace=True,axis=1)

True     22677
False      451
Name: Free, dtype: int64
Apps that have Price = 0, have Free column True
True     22678
False      450
Name: Free, dtype: int64


In [23]:
df['RatingType'] = 'NoRating'
df.loc[(df['RatingCount'] > 0) & (df['RatingCount'] <= 10000.0),'RatingType'] = 'Less than 10K'
df.loc[(df['RatingCount'] > 10000) & (df['RatingCount'] <= 500000.0),'RatingType'] = 'Between 10K and 500K'
df.loc[(df['RatingCount'] > 500000) & (df['RatingCount'] <= 138557570.0),'RatingType'] = 'More than 500K'
print(df.RatingType.value_counts())


df['InstallRange'] = 'NoInstalls'
df.loc[(df['Installs'] > 0) & (df['Installs'] <= 10000.0),'InstallRange'] = 'Less than 10K'
df.loc[(df['Installs'] > 10000) & (df['Installs'] <= 500000.0),'InstallRange'] = 'Between 10K and 500K'
df.loc[(df['Installs'] > 500000) & (df['Installs'] <= 138557570.0),'InstallRange'] = 'More than 500K'
print(df.InstallRange.value_counts())

Less than 10K           12222
NoRating                10557
Between 10K and 500K      328
More than 500K             21
Name: RatingType, dtype: int64
Less than 10K           20443
Between 10K and 500K     2132
More than 500K            454
NoInstalls                 99
Name: InstallRange, dtype: int64


In [24]:
labelEncoder = LabelEncoder()

labelEncoder.fit(df['Category'])
df['Category'] = labelEncoder.transform(df['Category'])
labelEncoder.fit(df['Type'])
df['Type'] = labelEncoder.transform(df['Type'])
labelEncoder.fit(df['ContentRating'])
df['ContentRating'] = labelEncoder.transform(df['ContentRating'])
labelEncoder.fit(df['AdSupported'])
df['AdSupported'] = labelEncoder.transform(df['AdSupported'])
labelEncoder.fit(df['EditorsChoice'])
df['EditorsChoice'] = labelEncoder.transform(df['EditorsChoice'])
labelEncoder.fit(df['InAppPurchases'])
df['InAppPurchases'] = labelEncoder.transform(df['InAppPurchases'])
labelEncoder.fit(df['RatingType'])
df['RatingType'] = labelEncoder.transform(df['RatingType'])
labelEncoder.fit(df['PriceRange'])
df['PriceRange'] = labelEncoder.transform(df['PriceRange'])
labelEncoder.fit(df['InstallRange'])
df['RatingType'] = labelEncoder.transform(df['InstallRange'])

print(df.head(10))

                                                   AppName  Category  Rating  \
128037                       Burn it Down Piano Cover Song        17     0.0   
491755                                        أنعام الوسطى        20     4.3   
491263                           Block Puzzle Ultimate Fun        34     0.0   
836489   Learning Professions and Occupations for Toddlers        16     3.7   
73349                                             Finutive         8     4.3   
1166159                   St. Thomas of Canterbury College        42     0.0   
413825                                   Dubai Super Store        37     4.4   
925255                           ◆シニア向け◆　ボケ防止のためのクイズゲーム 雑学        15     0.0   
1215878                       Stockton R-1 School District        15     4.6   
1070334                                     Miind Changers        15     0.0   

         RatingCount  Installs  MinimumInstalls  MaximumInstalls  Price  Size  \
128037           0.0      10.0        